### setup

In [1]:
%load_ext lab_black

In [2]:
import requests
import urllib.request
import numpy as np
import datetime
import random

In [3]:
import pandas as pd

# import gspread
# from oauth2client.service_account import ServiceAccountCredentials

In [4]:
import altair as alt
import altair_stiles as altstiles

alt.themes.register("stiles", altstiles.theme)
alt.themes.enable("stiles")

ThemeRegistry.enable('grid')

In [5]:
from datawrapper import Datawrapper

dw = Datawrapper(
    access_token="FtIwtvFtoGLaRT9a3gjX69PLu4wSuRyKddoOz6SOPw3k9wWyNICMHTkcPhOGCR5Z"
)

In [6]:
# scope = ["https://spreadsheets.google.com/feeds"]
# credentials = ServiceAccountCredentials.from_json_keyfile_name(
#    "jupyter-integration-credentials.json", scope
# )
# gc = gspread.authorize(credentials)

In [7]:
# spreadsheet_key = "1_RXzXkHPEyDAiDKmz98wTSC9UNo-8OyjGTT540X_vJk"
# book = gc.open_by_key(spreadsheet_key)

In [8]:
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000

In [9]:
# Opening the worksheet by using Worksheet ID
# workbook = gc.open_by_key(spreadsheet_key)
# Selecting which sheet to pulling the data
# sheet = workbook.worksheet("Sheet1")
# Pulling the data and transform it to the data frame
# values = sheet.get_all_values()

#### read in data

In [10]:
raw = pd.read_csv("data/raw/NCHS_quarterly_provisional.csv")

In [11]:
raw.columns = raw.columns.str.lower().str.replace(" ", "_")

In [12]:
df = raw[
    (raw["time_period"] == "12 months ending with quarter")
    & (raw["rate_type"] == "Crude")
    & (raw["cause_of_death"] != "All causes")
].drop(list(raw)[8:69], axis=1)

In [13]:
df["date"] = (
    df["year_and_quarter"]
    .str.replace(" Q1", "-03-31")
    .str.replace(" Q2", "-06-30")
    .str.replace(" Q3", "-09-30")
    .str.replace(" Q4", "-12-31")
)

In [14]:
df["date"] = pd.to_datetime(df["date"])

looking at top five causes over time

In [15]:
top_five = (
    df.sort_values(["date", "overall_rate"], ascending=[True, False])
    .groupby(["date"])
    .head(5)
)

In [16]:
causes = pd.Series(
    top_five.value_counts("cause_of_death").reset_index()["cause_of_death"]
)

In [17]:
top_causes = df[(df["cause_of_death"].isin(causes)) & (~df["overall_rate"].isna())][
    ["date", "cause_of_death", "overall_rate"]
]

In [18]:
top_causes["rate"] = top_causes["overall_rate"].astype(float)

In [19]:
alt.Chart(top_causes[top_causes["cause_of_death"] != "COVID-19"]).mark_line().encode(
    x="date", y="rate", color="cause_of_death:N"
)

alt.Chart(...)

### Here's where I might wanna play with percent change but it doesn't make sense to do it for every column in the wide format 

In [23]:
alt.Chart(top_causes[top_causes["cause_of_death"] != "COVID-19"]).mark_line().encode(
    x=alt.X("date"), y=alt.Y("rate")
).properties(width=120, height=120).facet(
    facet=alt.Facet("cause_of_death:O"), columns=4
)

alt.FacetChart(...)

don't know what to try here